# agof Use Case

In this use case, we will fetch, process and extract some preliminary results from the 'daily digital facts' market media study by agof (Arbeitsgemeinschaft Online Forschung).

This notebook is divided into three parts:
1. Downloading Raw Data
2. Processing Downloaded Data
3. Extracting Facts

You are encouraged to play around. The blocks with comments beginning in `# note: ...` or `# sanity check: ...` have been included to further your understanding of the logic and to demonstrate how a data scientist will go about working on this problem. These blocks are not vital to the core logic of this use case and can therefore be skipped when you translate the code from this notebook to core4 jobs.

Lines of code which produce a large output or which have been commented out with three hash signs (`###`) for better readability. You can uncomment them in order to run them.

**A note on python notebooks:** Python notebooks such as this one are flexible in that you can run your code bit by bit. The downside is that you have the possibility to run blocks of code in an arbitrary order. We have divided the code in this notebook into several blocks to explain it more easily. Please make sure you run them in the order in which we have already written them. This also applies when you translate this code into core4 jobs as a part of your assignment.

### 1. Downloading Raw Data

In [ ]:
# fetching the webpage

import requests

url = "https://www.agof.de/service-downloads/downloadcenter/download-daily-digital-facts/"
rv = requests.get(url)
body = rv.content.decode("utf-8")

In [ ]:
# sanity check: how is the content we fetched stored by python?

type(body)

In [ ]:
# sanity check: have we fetched the right thing?

### body

In [ ]:
# scraping the fetched content

from bs4 import BeautifulSoup

soup = BeautifulSoup(body, "html.parser")
tables_list = soup.find_all("tr")

In [ ]:
# sanity check: have we scraped correctly? (1/2)

tables_list[1]

In [ ]:
# sanity check: have we scraped correctly? (2/2)

tables_list[1].text

In [ ]:
# isolating relevant links from the list of scraped html table rows (<tr>...</tr>)

links = [item for item in tables_list if "Angebote Ranking" in item.text]
links_list = [item for item in links if "xlsx" in item.text]

In [ ]:
# sanity check: did we isolate the right links?

str(links_list[0])

In [ ]:
# using regular expresssions to extract the link from each string in the list

import re
re.findall("href=[\"\'](.+?)[\"\']", str(links_list[0]))

In [ ]:
xls = []
for i in links_list:
    xls.append(re.findall("href=[\"\'](.+?)[\"\']", str(i))[0])

In [ ]:
# sanity check: does our list of links look right? (1/3)

### xls

In [ ]:
# sanity check: does our list of links look right? (2/3)

xls[0]

In [ ]:
# sanity check: does our list of links look right? (3/3)

len(xls)

### 2. Processing Downloaded Data

In [ ]:
import pandas as pd

In [ ]:
# sanity check: process just the first excel to begin with (1/7)

rv = requests.get(xls[0])
open("/tmp/test.xlsx", "wb").write(rv.content)

In [ ]:
# sanity check: process just the first excel to begin with (2/7)

# read data without skipping rows and find out where the actual data starts (the first few rows can be metadata)
df = pd.read_excel("/tmp/test.xlsx", header=None)
df.head(15)

In [ ]:
# note:
# it also possible to read the excel such that we skip the metadata from the very beginning
# but in our case, are interested in saving the metadata, so we will NOT be doing this

# if you are curious, this is how we would have skipped the rows containing metadata:
df_no_metadata = pd.read_excel("/tmp/test.xlsx", skiprows = 8)
### df_no_metadata.head()

In [ ]:
# sanity check: process just the first excel to begin with (3/7)

# save metadata in separate variables
# in python, "assert" raises an error if the condition you pass it is false
assert df.iloc[0, 0] == "Analyse"
analyse = df.iloc[0, 1]
assert df.iloc[1, 0] == "Grundgesamtheit"
grundgesamtheit = df.iloc[1, 1]
assert df.iloc[2, 0] == "Zeitraum"
zeitraum = df.iloc[2, 1]
assert df.iloc[3, 0] == "Vorfilter"
vorfilter = df.iloc[3, 1]
vorfilter_fallzahl = df.iloc[4, 1]
assert df.iloc[5, 0] == "Zielgruppe"
zielgruppe = df.iloc[5, 1]
zielgruppe_fallzahl = df.iloc[6, 1]

In [ ]:
# sanity check: process just the first excel to begin with (4/7)

# identify where the data starts
ln = 7
while df.iloc[ln, 0] != "Basis":
    ln += 1
    if ln > 1000:
        raise  RuntimeError("failed to identify start of data")
        
ln

In [ ]:
# sanity check: process just the first excel to begin with (5/7)

# save the subset of 'df' containing the main data in a separeate dataframe, 'dframe'
dframe = df.iloc[ln:].copy()

dframe.head()

In [ ]:
# sanity check: process just the first excel to begin with (6/7)

# extract column names from 'df' and save them as a list 'cols'
cols = list(df.iloc[ln-1])
cols[0] = "Titel"

# name the columns of df using the list 'cols'
dframe.columns = ["" if pd.isnull(c)
             else c.replace("\n", " ").replace(".", "") for c in cols]
if "" in dframe.columns:
    dframe.drop([""], axis=1, inplace=True)

# add columns containing the medtadata we saved earlier
dframe["Analyse"] = analyse
dframe["Grundgesamtheit"] = grundgesamtheit
dframe["Zeitraum"] = zeitraum
dframe["Vorfilter"] = vorfilter
dframe["Zielgruppe"] = zielgruppe

dframe.head()

In [ ]:
# sanity check: process just the first excel to begin with (7/7)

# check how many different time periods ('Zeitraum') are covered in 'dframe'
dframe.Zeitraum.value_counts()

In [ ]:
# a function that generalizes what we did above with one excel
# this function can later be called for any of the excels in 'xls'

def process(df):
    assert df.iloc[0, 0] == "Analyse"
    analyse = df.iloc[0, 1]
    assert df.iloc[1, 0] == "Grundgesamtheit"
    grundgesamtheit = df.iloc[1, 1]
    assert df.iloc[2, 0] == "Zeitraum"
    zeitraum = df.iloc[2, 1]
    assert df.iloc[3, 0] == "Vorfilter"
    vorfilter = df.iloc[3, 1]
    vorfilter_fallzahl = df.iloc[4, 1]
    assert df.iloc[5, 0] == "Zielgruppe"
    zielgruppe = df.iloc[5, 1]
    zielgruppe_fallzahl = df.iloc[6, 1]
    ln = 7
    while df.iloc[ln, 0] != "Basis":
        ln += 1
        if ln > 1000:
            raise  RuntimeError("failed to identify start of data")
    d = df.iloc[ln:].copy()
    cols = list(df.iloc[ln-1])
    cols[0] = "Titel"
    d.columns = ["" if pd.isnull(c)
                 else c.replace("\n", " ").replace(".", "") for c in cols]
    if "" in d.columns:
        d.drop([""], axis=1, inplace=True)
    d["Analyse"] = analyse
    d["Grundgesamtheit"] = grundgesamtheit
    d["Zeitraum"] = zeitraum
    d["Vorfilter"] = vorfilter
    d["Zielgruppe"] = zielgruppe
    return(d)

In [ ]:
# create a list to store the data generated after processing excel files
# save processed data from xls[0] in it
fin_df = list()
fin_df.append(dframe)

# process several excels from 'xls' using the 'process()' function we defined above
# we process 29 files in this case (range(1,30))
# we start 'i' from 1 as we have already added processed data from xls[0] to the list 'fin_df'
for i in range(1,30):
    rv = requests.get(xls[i])
    open("/tmp/test"+str(i)+".xlsx", "wb").write(rv.content)
    df = pd.read_excel("/tmp/test"+str(i)+".xlsx", header=None)
    df_processed = process(df)
    fin_df.append(df_processed)

# saving 'fin_df' as a dataframe    
fin_df = pd.concat(fin_df)

In [ ]:
# sanity check: what does 'fin_df' look like now? (1/2)

fin_df.head()

In [ ]:
# sanity check: what does 'fin_df' look like now? (2/2)

fin_df.shape

In [ ]:
# sanity check: how many different time periods ()'Zeitraum') are covered in 'fin_df'

fin_df.Zeitraum.value_counts()

### Extracting Facts

In [ ]:
import datetime

MONAT = {
    "Januar": "01",
    "Februar": "02",
    "März": "03",
    "April": "04",
    "Mai": "05",
    "Juni": "06",
    "Juli": "07",
    "August": "08",
    "September": "09",
    "Oktober": "10",
    "November": "11",
    "Dezember": "12"
}

monat = fin_df.Zeitraum.apply(lambda s: s.replace("Letzter Monat (", "").replace(")", "").split())

In [ ]:
# sanity check: what data type is 'monat' stored as?

type(monat)

In [ ]:
# sanity check: what does 'monat' look like?

monat.head()

In [ ]:
# adding columns to 'fin_df'

fin_df["Monat"] = [datetime.datetime.strptime("01." + MONAT[m[0]] + "." + m[1], "%d.%m.%Y") for m in monat]
fin_df["val"] = fin_df["Kontakte Mio"].apply(pd.to_numeric, errors='coerce')
fin_df['Date'] = fin_df.Monat.apply(lambda x: x.date().isoformat())

In [ ]:
# sanity check: list the columns in 'fin_df'

fin_df.columns

In [ ]:
# sanity check: check the values different categorical variables take on (here, the variable 'Medientyp')

fin_df.Medientyp.unique()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib notebook

fin_df = fin_df.replace(np.nan,0)
g = fin_df.groupby(["Date"]).val.sum()
g.plot.bar()
plt.ylabel("Contacts")
plt.tight_layout()

In [ ]:
%matplotlib notebook

df_new = fin_df[fin_df.Medientyp != 0]
g1 = df_new.groupby(["Medientyp"]).val.sum()
g1.plot.bar()
plt.ylabel("Contacts")
plt.xticks(rotation='horizontal')
plt.tight_layout()

In [ ]:
# sanity check

list(g1.index)

In [ ]:
%matplotlib notebook

df_new = fin_df[fin_df.Medientyp != 0]
# Monthly contacts for each media group
g1 = df_new.groupby(["Date","Medientyp"]).val.sum().unstack()
# contact of different media group per month
# g1 = df_new.groupby(["Date","Medientyp"]).val.sum().unstack(0)
plt.rcParams["figure.figsize"] = [7,7]
g1.plot.bar(rot=45)

plt.ylabel("Contacts")
plt.legend(fontsize='small')
plt.tight_layout()